**This Jupyter Notebook ask for a Cellpose model, then a folder containing images, and segment the images from this folder using the model**
<p>Then, it computes the size of each mask and in particular the diameter, and export the value for each images, with the suffix *_data*</p>
<p>Then, it merges all the data from one image folder into one file with the suffix *data_pos*</p>
<p>Once you are done computing all the image folders, it merges all the data from one image folder into one file with the suffix *data_pos*</p>

In [5]:
# Import required libraries
import glob
import os
from pathlib import Path
import tkinter

import matplotlib.pyplot as plt
import seaborn as sns
#import ptitprince as pt # if not installed, run: "conda install -c conda-forge ptitprince"

import numpy as np
from tkinter.filedialog import askdirectory, askopenfilename
import pandas as pd
from skimage import io, measure
from cellpose import models, utils, io as iocp

import ipywidgets as widgets

In [9]:
# Create a button widget
button = widgets.Button(description="Find path to the model",
                        layout={'height': 'auto', 'width': 'auto'})
# Create a text output widget
output = widgets.Text(layout={'height': 'auto', 'width': '100%'})

# Define a function to be called when the button is clicked
def find_path(b):
    # Use the filedialog module to open a file or directory selection dialog
    root = tkinter.Tk()
    root.withdraw()
    model_path= askopenfilename(title="Path to the model of segmentation")
    output.value=model_path

# Attach the button-click function to the button widget
button.on_click(find_path)

# Display the button widget in the output area
display(widgets.HBox(children=[button, output]))

In [10]:
#%%
pixsize=150/208 #in microns

# Create a text input widget
text_input = widgets.FloatText(value=pixsize,
                               layout={'height': 'auto', 'width': 'auto'})

# Define a function to be called when the text input is changed
def update_variable(change):
    # Update the value of the variable to the current value of the text input
    pixsize = change["new"]

# Attach the text input change function to the text input widget
text_input.observe(update_variable, names="value")

# Display the text input widget in the output area

labelpix = widgets.Label(value="Pixel size (in µm)")
containerpix = widgets.HBox(children=[labelpix, text_input])
display(containerpix)




timeu=3;

# Create a text input widget
text_inputtime = widgets.FloatText(value=timeu,
                                   layout={'height': 'auto', 'width': 'auto'})

# Define a function to be called when the text input is changed
def update_variable(change):
    # Update the value of the variable to the current value of the text input
    timeu = change["new"]

# Attach the text input change function to the text input widget
text_inputtime.observe(update_variable, names="value")

# Display the text input widget in the output area
labeltime = widgets.Label(value="Timepoint (in whatever unit you wish)")
containertime = widgets.HBox(children=[labeltime, text_inputtime])
display(containertime)






# Create a button widget
button = widgets.Button(description="Find path to the images", 
                        layout={'height': 'auto', 'width': 'auto'})
# Create a text output widget
output = widgets.Text(layout={'height': 'auto', 'width': '100%'})

# Define a function to be called when the button is clicked
def find_path(b):
    # Use the filedialog module to open a file or directory selection dialog
    root = tkinter.Tk()
    root.withdraw()
    folder_path = askdirectory(title="Path to the folder containing images to be segmented")
    output.value=folder_path

# Attach the button-click function to the button widget
button.on_click(find_path)

# Display the button widget in the output area
display(widgets.HBox(children=[button, output]))

In [ ]:
#%%
# Get the list of images in the folder
#image_paths =glob.glob(folder_path + "/*.jpg")
image_paths=iocp.get_image_files(folder_path, ['_masks'], imf=None, look_one_level_down=False)

image_paths=list(dict.fromkeys(image_paths)) #remove duplicates using a dictionary fct
# cellpose.io.add_model(filename)

# Load the cellpose model
model = models.CellposeModel(gpu=True, pretrained_model=model_path)

channels=[0,0]
cc=1;
# Segment the image using cellpose
for image_path in image_paths:
    img= io.imread(image_path)
    mask, flow, style = model.eval(img,channels=channels,diameter=150)

    iocp.save_masks(img, 
              mask, 
              flow, 
              image_path, 
              channels=channels,
              png=True, # save masks as PNGs and save example image
              tif=False, # save masks as TIFFs
              save_txt=False, # save txt outlines for ImageJ
              save_flows=False, # save flows as TIFFs
              save_outlines=False, # save outlines as TIFFs 
              )

    mask2=utils.remove_edge_masks(mask)
    outline=utils.masks_to_outlines(mask2)
    #perimeters=utils.get_mask_perimeters(mask2)
    regions = measure.regionprops(mask2)
    
    diameters=[]
    source_image=[]
    time=[]
    #perimeters2=np.empty((len(regions),1))
    for k in range(len(regions)):
        diameters+=[regions[k].equivalent_diameter_area]
        source_image+=[image_path]
        time+=[timeu]
        #perimeters2[k]=regions[k].perimeter

    data=pd.DataFrame({
        'time': timeu,
        'diameter': diameters,
        'source_image': source_image
        })
    
    #todo: one timepoint per row (per organoid)
    data.to_excel((os.path.splitext(image_path)[0] + '_data.xlsx'))
    data.to_pickle((os.path.splitext(image_path)[0] + '_data.pkl'))
# np.save((os.path.splitext(image_path)[0] + '_data'),diameters)
# np.savetxt((os.path.splitext(image_path)[0] + '_data.csv'),diameters)



#% Merge all the data from the position
data_paths =glob.glob(folder_path + "/*data.pkl")

data_pos=pd.DataFrame({
    'time': [],
    'diameter': [],
    'source_image': []
    })
# diameters_pos=pd.DataFrame([])
for data_path in data_paths:
    data = pd.read_pickle(data_path)
    # diameters=np.load(data_path)
    
    pos_name=os.path.basename(folder_path).split('_data')[0]
    
    data_pos=pd.concat([data_pos,data],ignore_index=True)
    # diameters_pos=np.append(diameters_all,diameters)

folder_all_path=str(Path(folder_path).parent.absolute())
data_pos.to_excel(os.path.join(folder_all_path, pos_name+'_data_pos.xlsx'))
data_pos.to_pickle(os.path.join(folder_all_path, pos_name+'_data_pos.pkl'))
    
# np.save((os.path.splitext(folder_path)[0] + "data_pos.npy"),diameters_pos)
# np.savetxt((os.path.splitext(folder_path)[0] + "data_pos.csv"),diameters_pos)

In [ ]:
#%%
# here print path in notebook and propose to change it with askdirectory
folder_all_path=str(Path(folder_path).parent.absolute())
data_all_paths =glob.glob(folder_all_path + "/*_data_pos.pkl")


data_all=pd.DataFrame({
    'time': [],
    'diameter': [],
    'source_image': []
    })
for data_pos_path in data_all_paths:
    data_pos = pd.read_pickle(data_pos_path)
    # diameters=np.load(data_path)
    data_all=pd.concat([data_all,data_pos],ignore_index=True)

folder_all_path=str(Path(folder_path).parent.absolute())
data_all.to_excel(os.path.join(folder_all_path, 'data_all.xlsx'))
data_all.to_pickle(os.path.join(folder_all_path, 'data_all.pkl'))

In [ ]:
#%%
#here load stuff
g = sns.catplot(data=data_all, x="time", y="diameter", kind="violin")

#set the alpha for the violin plot (no easier way...)
tmpcol=g.ax.collections[0].get_facecolor()
tmpcol[0,3]=0.5
for violin in g.ax.collections[:]:
    violin.set_facecolor(tmpcol)
    
#sns.swarmplot(data=data_all, x="time", y="diameter", color="k", size=3)
#plt.setp(g.ax.collections, alpha=.3)

plt.savefig(os.path.join(folder_all_path, 'data_all.pdf'))
plt.show()

**If you need to retrain the Cellpose model, first launch Cellpose** and follow the instructions from https://cellpose.readthedocs.io/en/latest/gui.html#training-your-own-cellpose-model

In [13]:
! python -m cellpose

2022-12-09 02:10:51,168 [INFO] WRITING LOG OUTPUT TO C:\Users\Silence\.cellpose\run.log
2022-12-09 02:10:52,097 [INFO] ** TORCH CUDA version installed and working. **
